In [ ]:
#!conda create -n MoveNet python=3.9 -y
#!conda run -n MoveNet pip install tensorflow tensorflow-hub opencv-python ipykernel
#!conda run -n MoveNet python -m ipykernel install --user --name=MoveNet --display-name "Python (MoveNet)"

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\chris\anaconda3\envs\MoveNet

  added / updated specs:
    - python=3.9


The following NEW packages will be INSTALLED:

  bzip2              pkgs/main/win-64::bzip2-1.0.8-h2bbff1b_6 
  ca-certificates    pkgs/main/win-64::ca-certificates-2025.2.25-haa95532_0 
  expat              pkgs/main/win-64::expat-2.7.1-h8ddb27b_0 
  libffi             pkgs/main/win-64::libffi-3.4.4-hd77b12b_1 
  openssl            pkgs/main/win-64::openssl-3.0.16-h3f729d1_0 
  pip                pkgs/main/noarch::pip-25.1-pyhc872135_2 
  python             pkgs/main/win-64::python-3.9.23-h716150d_0 
  setuptools         pkgs/main/win-64::setuptools-78.1.1-py39haa95532_0 
  sqlite             pkgs/main/win-64::sqlite-3.45.3-h2bbff1b_0 
  tk                 pkgs/main/win-64::tk-8.6.14-h5e9d12e_1 
  tzdata             pkgs/main/noarch::tzdata-2025b-h04d1e81_0 
  vc                 pkgs/main/win-64::vc-14.42-haa95532_5 
  



==> WARNING: A newer version of conda exists. <==
  current version: 23.1.0
  latest version: 25.5.1

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=25.5.1











  Using cached ipykernel-6.29.5-py3-none-any.whl.metadata (6.3 kB)





  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)








  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)


  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)







  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)




  Using cached typing_extensions-4.14.0-py3-none-any.whl.metadata (3.0 kB)









  Using cached numpy-2.0.2-cp39-cp39-win_amd64.whl.metadata (59 kB)




  Using cached ml_dtypes-0.5.1-cp39-cp39-win_amd64.whl.metadata (22 kB)

















  Using cached comm-0.2.2-py3-none-any.whl.metadata (3.7 kB)


  Using cached debugpy-1.8.14-cp39-cp39-win_amd64.whl.metadata (1.4 kB)


  Using cached ipython-8.18.1-py3-none-any.whl.metadata (6.0 kB)


  Using cached jupyter_client-8.6.3-py3-none-any.whl.metadata (8.3 kB)


  Using cached jupyter_core-5.8.1-py3-none-any.whl.metadata (1.6 kB)


  Using cached matplotlib_inline-

ERROR: Exception:

Traceback (most recent call last):

  File "C:\Users\chris\anaconda3\envs\MoveNet\lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher

    yield

  File "C:\Users\chris\anaconda3\envs\MoveNet\lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read

    data = self._fp_read(amt) if not fp_closed else b""

  File "C:\Users\chris\anaconda3\envs\MoveNet\lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read

    return self._fp.read(amt) if amt is not None else self._fp.read()

  File "C:\Users\chris\anaconda3\envs\MoveNet\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 98, in read

    data: bytes = self.__fp.read(amt)

  File "C:\Users\chris\anaconda3\envs\MoveNet\lib\http\client.py", line 463, in read

    n = self.readinto(b)

  File "C:\Users\chris\anaconda3\envs\MoveNet\lib\http\client.py", line 507, in readinto

    n = self.fp.readinto(b)

  File "C:\Users\chris\anaconda3\envs\MoveNet\l

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cv2

# Load MoveNet Thunder model
model = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
movenet = model.signatures['serving_default']

# Display size (resizing only for window display)
DISPLAY_WIDTH = 960
DISPLAY_HEIGHT = 540

# Optional: edge drawing definitions
KEYPOINT_EDGES = {
    (0, 1): 'm', (0, 2): 'm', (1, 3): 'm', (2, 4): 'm',
    (0, 5): 'g', (0, 6): 'g', (5, 7): 'g', (7, 9): 'g',
    (6, 8): 'g', (8, 10): 'g', (5, 6): 'y', (5, 11): 'g',
    (6, 12): 'g', (11, 12): 'y', (11, 13): 'g', (13, 15): 'g',
    (12, 14): 'g', (14, 16): 'g'
}

def draw_keypoints(frame, keypoints, confidence_threshold=0.3):
    h, w, _ = frame.shape
    keypoints = keypoints[0, 0, :, :]  # shape (17, 3)

    for i in range(17):
        y, x, c = keypoints[i]
        if c > confidence_threshold:
            cx, cy = int(x * w), int(y * h)
            cv2.circle(frame, (cx, cy), 4, (0, 255, 0), -1)

    for edge, color in KEYPOINT_EDGES.items():
        p1, p2 = edge
        y1, x1, c1 = keypoints[p1]
        y2, x2, c2 = keypoints[p2]
        if c1 > confidence_threshold and c2 > confidence_threshold:
            pt1 = (int(x1 * w), int(y1 * h))
            pt2 = (int(x2 * w), int(y2 * h))
            cv2.line(frame, pt1, pt2, (0, 255, 255), 2)

# Load video
cap = cv2.VideoCapture("snatch_video.mp4")  # or 'clean_helland.mp4'

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Prepare frame for model (256x256)
    input_image = cv2.resize(frame, (256, 256))
    input_image = tf.convert_to_tensor(input_image, dtype=tf.uint8)
    input_image = tf.expand_dims(input_image, axis=0)

    # Run pose estimation
    outputs = movenet(input_image)
    keypoints = outputs['output_0'].numpy()

    # Draw pose on original frame
    draw_keypoints(frame, keypoints)

    # Resize for display (preserve aspect ratio)
    h, w = frame.shape[:2]
    aspect_ratio = w / h
    if w > h:
        new_w = DISPLAY_WIDTH
        new_h = int(DISPLAY_WIDTH / aspect_ratio)
    else:
        new_h = DISPLAY_HEIGHT
        new_w = int(DISPLAY_HEIGHT * aspect_ratio)

    resized_frame = cv2.resize(frame, (new_w, new_h))
    cv2.imshow("MoveNet Thunder Pose Estimation", resized_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
